# Build a Chatbot

Based on [**this LangChain tutorial**](https://python.langchain.com/v0.2/docs/tutorials/chatbot/)

# Overview

We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:
- [**Conversational RAG**](https://python.langchain.com/v0.2/docs/tutorials/qa_chat_history/): Enable a chatbot experience over an external source of data.
- [**Agent**](https://python.langchain.com/v0.2/docs/tutorials/agents/): Build a chatbot that can take actions.

This tutorial will cover the basics which will be helpful for those two more advanced topics.

# Concepts

Here are a few of the high-level components we will be working with:
- [**Chat Models**](https://python.langchain.com/v0.2/docs/concepts/#chat-models). The chatbot interface is based around messages rather than raw text, and therefore is best suited to Chat Models rather than text LLMs.
- [**Prompt Templates**](https://python.langchain.com/v0.2/docs/concepts/#prompt-templates), which simplify the process of assembling prompts that combine:
    - default messages,
    - user input,
    - chat history,
    - (optionally) additional retrieved content
- [**Chat History**](https://python.langchain.com/v0.2/docs/concepts/#chat-history), which allows a chatbot to "remember" past interactions and take them into account, when responding to follow-up questions
- Debugging and tracking your application using [**LangSmith**](https://python.langchain.com/v0.2/docs/concepts/#langsmith)

We'll cover how to fit the above components together to create a powerful conversational chatbot.

# Setup

In [1]:
from dotenv import load_dotenv

In [2]:
_ = load_dotenv()

In [3]:
# To allow pretty printing
from rich import print as rprint

# Quickstart

In [4]:
from langchain_openai import ChatOpenAI

In [5]:
model = ChatOpenAI(model="gpt-3.5-turbo")

In [7]:
rprint(model)

ChatOpenAI(
    client=<openai.resources.chat.completions.Completions object at 0x76af8711d190>,
    async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x76af87551bd0>,
    openai_api_key=SecretStr('**********'),
    openai_proxy=''
)

Let's first use the model directly.

`ChatModel`s are instances of LangChain *Runnables*

In [11]:
model.__class__.__mro__

(langchain_openai.chat_models.base.ChatOpenAI,
 langchain_openai.chat_models.base.BaseChatOpenAI,
 langchain_core.language_models.chat_models.BaseChatModel,
 langchain_core.language_models.base.BaseLanguageModel,
 langchain_core.runnables.base.RunnableSerializable,
 langchain_core.load.serializable.Serializable,
 pydantic.v1.main.BaseModel,
 pydantic.v1.utils.Representation,
 langchain_core.runnables.base.Runnable,
 typing.Generic,
 abc.ABC,
 object)


This means **they expose a [standard interface](https://python.langchain.com/v0.1/docs/expression_language/interface/) for interacting with them**.

To just simply call the model, we can pass in a list of messages to the `.invoke` method.

In [12]:
from langchain_core.messages import HumanMessage

In [14]:
rprint(model.invoke([HumanMessage(content="Hi! I'm Bob.")]))

AIMessage(
    content='Hello Bob! How can I assist you today?',
    response_metadata={
        'token_usage': {'completion_tokens': 10, 'prompt_tokens': 13, 'total_tokens': 23},
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run-fc74b463-c858-4170-98cf-b04d22be32fb-0',
    usage_metadata={'input_tokens': 13, 'output_tokens': 10, 'total_tokens': 23}
)

> **API Reference: [HumanMessage](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.human.HumanMessage.html)**

The model, on its own, **does not have any concept of state**.

For example, if you ask a followup question like this:

In [15]:
rprint(model.invoke([HumanMessage(content="What's my name?")]))

AIMessage(
    content="I'm sorry, but I do not have access to personal information such as your name. Is there anything else 
I can help you with?",
    response_metadata={
        'token_usage': {'completion_tokens': 28, 'prompt_tokens': 12, 'total_tokens': 40},
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run-4ddf4dfa-9fae-4720-8638-142ad6894a20-0',
    usage_metadata={'input_tokens': 12, 'output_tokens': 28, 'total_tokens': 40}
)

Let's take a look at this example [**LangSmith trace**](https://smith.langchain.com/public/5c21cb92-2814-4119-bae9-d02b8db577ac/r)

We can see that **it doesn't take the previous conversation turn into context, and then cannot answer the last question**.

This makes for a terrible chatbot experience!

To get aroung this, we need to **pass the entire conversation history into the model**.

Let's see what happens in case we do that.

In [16]:
from langchain_core.messages import AIMessage

In [17]:
rprint(
    model.invoke([
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ])
)

AIMessage(
    content='Your name is Bob.',
    response_metadata={
        'token_usage': {'completion_tokens': 5, 'prompt_tokens': 35, 'total_tokens': 40},
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run-aac8e5e9-d2ee-4482-9a3b-391145cc70cf-0',
    usage_metadata={'input_tokens': 35, 'output_tokens': 5, 'total_tokens': 40}
)

And now we can see that we get a good response!

This is the basic idea underpinning a chatbot's ability to interact conversationnaly.

So, **how do we best implement this**?

# Message History

## Concepts

We can **use a Message History class to wrap our model and make it stateful**.

This will:
- **keep track of inputs and outputs of the model**,
- **store them in some datastore**.

Future interactions will then:
- **load those messages**,
- **pass them into the chain as part of the input**.

Let's see how to use this!

## Implementation

We can:
- import the relevant classes,
- set up our chain which wraps the model and adds in this message history.

**A key part here is the function we pass into as the `get_session_history`**.

This function **is expected to**:
- take in a `session_id`
- return a Message History object.

This `session_id`:
- is used to **distinguish between separate conversations**,
- should be **passed in as part of the config when calling the new chain**. (We'll show how to do that) 

In [20]:
from langchain_community.chat_message_histories import ChatMessageHistory  # Will be stored
from langchain_core.chat_history import BaseChatMessageHistory  # Only used for type hint
from langchain_core.runnables.history import RunnableWithMessageHistory  # Will be used to instanciate the wrapper

In [21]:
# Dict which will store the session_ids as keys
# and corresponding chat messages as values
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    """Chat history retriever, given a session_id"""
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [22]:
type(with_message_history)

langchain_core.runnables.history.RunnableWithMessageHistory

In [24]:
with_message_history.__class__.__mro__

(langchain_core.runnables.history.RunnableWithMessageHistory,
 langchain_core.runnables.base.RunnableBindingBase,
 langchain_core.runnables.base.RunnableSerializable,
 langchain_core.load.serializable.Serializable,
 pydantic.v1.main.BaseModel,
 pydantic.v1.utils.Representation,
 langchain_core.runnables.base.Runnable,
 typing.Generic,
 abc.ABC,
 object)

> **API Reference**
- [**ChatMessageHistory**](https://api.python.langchain.com/en/latest/chat_history/langchain_core.chat_history.ChatMessageHistory.html)
- [**BaseChatMessageHistory**](https://api.python.langchain.com/en/latest/chat_history/langchain_core.chat_history.BaseChatMessageHistory.html)
- [**RunnableWithMessageHistory**](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html)

> **IMPORTANT**
> 
> **We now need to create a `config` that we pass into the runnable every time**.

This config contains information that is not part of the input directly, but is still useful.

In this case, we want to include a `session_id`. This should look like...

In [25]:
config = {"configurable": {"session_id": "ObiwanKenobi"}}

In [27]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob.")],
    config=config,
)
response.content

'Hello Bob, nice to meet you! How are you doing today?'

In [28]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)
response.content

'Your name is Bob.'

Great! Our chatbot is now able to "remember" things about us.

If we change the config to reference a different `session_id`, we can start a conversation from fresh.

In [29]:
config = {"configurable": {"session_id": "Luke"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)
response.content

"I'm sorry, I do not have the ability to know your name."

However, we can still go back to the original conversation, as we're persisting it in `store`.

In [30]:
config = {"configurable": {"session_id": "ObiwanKenobi"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)
response.content

'Your name is Bob.'

This is how we can support a chatbot having conversations with many users!

Right now, all we-ve done is add a simple persistence layer around the model.

We can start to make the more complicated and personalized by adding in a **prompt template**.

# Prompt Templates

**Prompt templates** help to **turn raw user information into a format that the LLM can work with**.

In this case, the raw user input is just a message, that we are passing to the LLM.

Let's now make it a bit more complicated.
- First, let's add in a **system message** with some custom instructions (but still taking messages as input).
- Next, we'll add in more input besides just as messages.

In order to do this, we will:
- Create a `ChatPromptTemplate`, and through its `from_messages` method pass a list of messages:
- first, a **system message** we will define,
- then, we will make use of the `MessagePlaceholder` class to pass all the messages in.

In [32]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [33]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all questions to the best of your ability."),
    MessagesPlaceholder(variable_name="messages")
])

In [35]:
type(prompt)

langchain_core.prompts.chat.ChatPromptTemplate

> **NOTE**
>
> The prompt objects have a `pretty_print` method that helps display their internals.

In [34]:
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant. Answer all questions to the best of your ability.

============================= Messages Placeholder =============================

{messages}


> **API Reference**
- [**ChatPromptTemplate**](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html)
- [**MessagesPlaceholder**](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.MessagesPlaceholder.html)

Let's chain all prompt and model.

In [36]:
chain = prompt | model

In [37]:
type(chain)

langchain_core.runnables.base.RunnableSequence

> **NOTE**
> 
> This slightly changes the input type.

Rather than pass in a list of messages, we now have to pass a `dict` with a `messages` key, whose value is a `list` of messages.

In [40]:
rprint(
    chain.invoke({
        "messages": [HumanMessage(content="Hi! I'm Bob.")]
    })
)

AIMessage(
    content='Hello Bob! How can I assist you today?',
    response_metadata={
        'token_usage': {'completion_tokens': 10, 'prompt_tokens': 33, 'total_tokens': 43},
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run-16d12e0a-56d5-40cb-a6a9-0e16428fc19c-0',
    usage_metadata={'input_tokens': 33, 'output_tokens': 10, 'total_tokens': 43}
)

**In order to make chat history persist, we can wrap this the same way as before**.

In [41]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [42]:
config = {"configurable": {"session_id": "DarkVador"}}

In [44]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Jim.")],
    config = config
)
response.content

'Hello, Jim! How can I assist you today?'

> **NOTE**
> 
> Notice that **we've come back to our first way of passing messages within a list**.

In [45]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Jim.'

In [47]:
# Just to check the conversations store
rprint(store)

{
    'ObiwanKenobi': InMemoryChatMessageHistory(
        messages=[
            HumanMessage(content="Hi! I'm Bob."),
            AIMessage(
                content='Hello Bob! How can I assist you today?',
                response_metadata={
                    'token_usage': {'completion_tokens': 10, 'prompt_tokens': 13, 'total_tokens': 23},
                    'model_name': 'gpt-3.5-turbo',
                    'system_fingerprint': None,
                    'finish_reason': 'stop',
                    'logprobs': None
                },
                id='run-e361fb7c-6eaa-45c5-a38e-c873abdfbb6f-0',
                usage_metadata={'input_tokens': 13, 'output_tokens': 10, 'total_tokens': 23}
            ),
            HumanMessage(content="Hi! I'm Bob."),
            AIMessage(
                content='Hello Bob, nice to meet you! How are you doing today?',
                response_metadata={
                    'token_usage': {'completion_tokens': 14, 'prompt_tokens': 37, 'total_tokens': 51},
                    'model_name': 'gpt-3.5-turbo',
                    'system_fingerprint': None,
                    'finish_reason': 'stop',
                    'logprobs': None
                },
                id='run-2ecb05a8-4617-4287-9138-072f1a1d9dfb-0',
                usage_metadata={'input_tokens': 37, 'output_tokens': 14, 'total_tokens': 51}
            ),
            HumanMessage(content="What's my name?"),
            AIMessage(
                content='Your name is Bob.',
                response_metadata={
                    'token_usage': {'completion_tokens': 5, 'prompt_tokens': 64, 'total_tokens': 69},
                    'model_name': 'gpt-3.5-turbo',
                    'system_fingerprint': None,
                    'finish_reason': 'stop',
                    'logprobs': None
                },
                id='run-7facfcd1-1a3f-4591-b54f-a28f2e3e7b8e-0',
                usage_metadata={'input_tokens': 64, 'output_tokens': 5, 'total_tokens': 69}
            ),
            HumanMessage(content="What's my name?"),
            AIMessage(
                content='Your name is Bob.',
                response_metadata={
                    'token_usage': {'completion_tokens': 5, 'prompt_tokens': 82, 'total_tokens': 87},
                    'model_name': 'gpt-3.5-turbo',
                    'system_fingerprint': None,
                    'finish_reason': 'stop',
                    'logprobs': None
                },
                id='run-7990b98a-c0db-4b42-9ef0-778a1ef17566-0',
                usage_metadata={'input_tokens': 82, 'output_tokens': 5, 'total_tokens': 87}
            )
        ]
    ),
    'Luke': InMemoryChatMessageHistory(
        messages=[
            HumanMessage(content="What's my name?"),
            AIMessage(
                content="I'm sorry, I do not have the ability to know your name.",
                response_metadata={
                    'token_usage': {'completion_tokens': 15, 'prompt_tokens': 12, 'total_tokens': 27},
                    'model_name': 'gpt-3.5-turbo',
                    'system_fingerprint': None,
                    'finish_reason': 'stop',
                    'logprobs': None
                },
                id='run-a321ce6c-f955-438d-b4d9-d67a89665745-0',
                usage_metadata={'input_tokens': 12, 'output_tokens': 15, 'total_tokens': 27}
            )
        ]
    ),
    'DarkVador': InMemoryChatMessageHistory(
        messages=[
            HumanMessage(content="Hi! I'm Jim."),
            AIMessage(
                content='Hello, Jim! How can I assist you today?',
                response_metadata={
                    'token_usage': {'completion_tokens': 11, 'prompt_tokens': 33, 'total_tokens': 44},
                    'model_name': 'gpt-3.5-turbo',
                    'system_fingerprint': None,
                    'finish_reason': 'stop',
                    'logprobs': None
                },
                id='run-43abcd7f-4108-4497-b19e

Awesome! Let's now make our prompt a little bit more complicated.

Let's assume that the prompt template now looks something like this:

In [48]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all questions to the best of your ability in {language}"),
    MessagesPlaceholder(variable_name="messages"),
])

chain = prompt | model

In [50]:
rprint(chain)

RunnableSequence(
    first=ChatPromptTemplate(
        input_variables=['language', 'messages'],
        input_types={
            'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, 
langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, 
langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, 
langchain_core.messages.tool.ToolMessage]]
        },
        messages=[
            SystemMessagePromptTemplate(
                prompt=PromptTemplate(
                    input_variables=['language'],
                    template='You are a helpful assistant. Answer all questions to the best of your ability in 
{language}'
                )
            ),
            MessagesPlaceholder(variable_name='messages')
        ]
    ),
    last=ChatOpenAI(
        client=<openai.resources.chat.completions.Completions object at 0x76af8711d190>,
        async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x76af87551bd0>,
        openai_api_key=SecretStr('**********'),
        openai_proxy=''
    )
)

Now, **we have include a new `language` input to the prompt (specified as `input_variables` in the previous display)**

We can now invoke the chain **and pass in a language of our choice**.

In [51]:
response = chain.invoke({
    "messages": [HumanMessage(content="Hi! I'm Bob.")],
    "language": "Spanish"
})

response.content

'¡Hola, Bob! ¿En qué puedo ayudarte hoy?'

Let's now wrap it in a Message History class.

> **IMPORTANT**
> 
> This time, **because there are multiple keys in the input, we need to specify the correct key to use to save the chat history**.

In [52]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [53]:
config = {"configurable": {"session_id": "Jarjar Bings"}}

In [54]:
response = with_message_history.invoke(
    {
        "messages": "Hi! I'm Todd!",
        "language": "Spanish",
    },
    config=config
)
response.content

'¡Hola Todd! ¿En qué puedo ayudarte hoy?'

In [55]:
response = with_message_history.invoke(
    {
        "messages": "What's my name?",
        "language": "Spanish",
    },
    config=config
)
response.content

'Tu nombre es Todd.'